In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox, scrolledtext
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')

class IndianCarRecommendationSystem:
    def __init__(self):
        self.cars_df = None
        self.scaler = StandardScaler()
        self.label_encoders = {}
        self.rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
        self.kmeans_model = None
        self.setup_sample_data()
        self.prepare_models()
        
    def setup_sample_data(self):
        """Create comprehensive Indian car dataset"""
        cars_data = {
            'car_id': range(1, 81),
            'brand': [
                'Maruti Suzuki', 'Maruti Suzuki', 'Maruti Suzuki', 'Maruti Suzuki', 'Maruti Suzuki',
                'Hyundai', 'Hyundai', 'Hyundai', 'Hyundai', 'Hyundai',
                'Tata', 'Tata', 'Tata', 'Tata', 'Tata',
                'Mahindra', 'Mahindra', 'Mahindra', 'Mahindra', 'Mahindra',
                'Honda', 'Honda', 'Honda', 'Honda', 'Honda',
                'Toyota', 'Toyota', 'Toyota', 'Toyota', 'Toyota',
                'Kia', 'Kia', 'Kia', 'Kia', 'Kia',
                'MG', 'MG', 'MG', 'MG', 'MG',
                'Nissan', 'Nissan', 'Nissan', 'Nissan', 'Nissan',
                'Renault', 'Renault', 'Renault', 'Renault', 'Renault',
                'Ford', 'Ford', 'Ford', 'Ford', 'Ford',
                'Volkswagen', 'Volkswagen', 'Volkswagen', 'Volkswagen', 'Volkswagen',
                'Skoda', 'Skoda', 'Skoda', 'Skoda', 'Skoda',
                'BMW', 'BMW', 'BMW', 'BMW', 'BMW',
                'Audi', 'Audi', 'Audi', 'Audi', 'Audi',
                'Mercedes-Benz', 'Mercedes-Benz', 'Mercedes-Benz', 'Mercedes-Benz', 'Mercedes-Benz'
            ],
            'model': [
                'Alto 800', 'Swift', 'Baleno', 'Vitara Brezza', 'Ertiga',
                'i10 Nios', 'i20', 'Creta', 'Venue', 'Verna',
                'Tiago', 'Altroz', 'Nexon', 'Harrier', 'Safari',
                'Bolero', 'Scorpio', 'XUV300', 'XUV700', 'Thar',
                'Amaze', 'Jazz', 'City', 'WR-V', 'CR-V',
                'Glanza', 'Urban Cruiser', 'Yaris', 'Innova Crysta', 'Fortuner',
                'Sonet', 'Seltos', 'Carens', 'Carnival', 'EV6',
                'Hector', 'ZS EV', 'Astor', 'Gloster', 'Hector Plus',
                'Magnite', 'Kicks', 'GT-R', 'X-Trail', 'Patrol',
                'Kwid', 'Triber', 'Kiger', 'Duster', 'Captur',
                'Figo', 'Freestyle', 'Aspire', 'EcoSport', 'Endeavour',
                'Polo', 'Vento', 'Taigun', 'Tiguan', 'T-Roc',
                'Rapid', 'Octavia', 'Superb', 'Kodiaq', 'Kushaq',
                'X1', 'X3', '3 Series', '5 Series', '7 Series',
                'Q3', 'Q5', 'A4', 'A6', 'A8',
                'GLA', 'GLC', 'C-Class', 'E-Class', 'S-Class'
            ],
            'fuel_type': [
                'Petrol', 'Petrol', 'Petrol', 'Diesel', 'Petrol',
                'Petrol', 'Petrol', 'Petrol', 'Petrol', 'Diesel',
                'Petrol', 'Petrol', 'Petrol', 'Diesel', 'Diesel',
                'Diesel', 'Diesel', 'Petrol', 'Diesel', 'Petrol',
                'Petrol', 'Petrol', 'Petrol', 'Petrol', 'Petrol',
                'Petrol', 'Petrol', 'Petrol', 'Diesel', 'Diesel',
                'Petrol', 'Petrol', 'Petrol', 'Diesel', 'Electric',
                'Petrol', 'Electric', 'Petrol', 'Diesel', 'Petrol',
                'Petrol', 'Petrol', 'Petrol', 'Petrol', 'Petrol',
                'Petrol', 'Petrol', 'Petrol', 'Petrol', 'Petrol',
                'Petrol', 'Petrol', 'Petrol', 'Petrol', 'Diesel',
                'Petrol', 'Petrol', 'Petrol', 'Petrol', 'Petrol',
                'Petrol', 'Petrol', 'Petrol', 'Diesel', 'Petrol',
                'Petrol', 'Petrol', 'Petrol', 'Petrol', 'Petrol',
                'Petrol', 'Petrol', 'Petrol', 'Petrol', 'Petrol',
                'Petrol', 'Petrol', 'Petrol', 'Petrol', 'Petrol'
            ],
            'transmission': [
                'Manual', 'Manual', 'CVT', 'Manual', 'Manual',
                'Manual', 'Manual', 'Automatic', 'Manual', 'Manual',
                'Manual', 'Manual', 'Manual', 'Automatic', 'Automatic',
                'Manual', 'Manual', 'AMT', 'Automatic', 'Manual',
                'CVT', 'Manual', 'CVT', 'Manual', 'CVT',
                'Manual', 'Manual', 'CVT', 'Automatic', 'Automatic',
                'Manual', 'Automatic', 'Manual', 'Automatic', 'Automatic',
                'Manual', 'Automatic', 'CVT', 'Automatic', 'Manual',
                'CVT', 'Manual', 'Automatic', 'CVT', 'Automatic',
                'Manual', 'Manual', 'AMT', 'Manual', 'CVT',
                'Manual', 'Manual', 'Automatic', 'Automatic', 'Automatic',
                'Manual', 'Manual', 'Automatic', 'Automatic', 'Automatic',
                'Manual', 'Automatic', 'Automatic', 'Automatic', 'Automatic',
                'Automatic', 'Automatic', 'Automatic', 'Automatic', 'Automatic',
                'Automatic', 'Automatic', 'Automatic', 'Automatic', 'Automatic',
                'Automatic', 'Automatic', 'Automatic', 'Automatic', 'Automatic'
            ],
            'body_type': [
                'Hatchback', 'Hatchback', 'Hatchback', 'SUV', 'MPV',
                'Hatchback', 'Hatchback', 'SUV', 'SUV', 'Sedan',
                'Hatchback', 'Hatchback', 'SUV', 'SUV', 'SUV',
                'SUV', 'SUV', 'SUV', 'SUV', 'SUV',
                'Sedan', 'Hatchback', 'Sedan', 'SUV', 'SUV',
                'Hatchback', 'SUV', 'Sedan', 'MPV', 'SUV',
                'SUV', 'SUV', 'MPV', 'MPV', 'SUV',
                'SUV', 'SUV', 'SUV', 'SUV', 'SUV',
                'SUV', 'SUV', 'Sports', 'SUV', 'SUV',
                'Hatchback', 'MPV', 'SUV', 'SUV', 'SUV',
                'Hatchback', 'SUV', 'Sedan', 'SUV', 'SUV',
                'Hatchback', 'Sedan', 'SUV', 'SUV', 'SUV',
                'Sedan', 'Sedan', 'Sedan', 'SUV', 'SUV',
                'SUV', 'SUV', 'Sedan', 'Sedan', 'Sedan',
                'SUV', 'SUV', 'Sedan', 'Sedan', 'Sedan',
                'SUV', 'SUV', 'Sedan', 'Sedan', 'Sedan'
            ],
            'price_lakhs': [
                3.5, 6.5, 7.5, 11.0, 9.5,
                5.5, 7.5, 12.5, 8.5, 10.5,
                5.0, 6.5, 8.5, 15.5, 17.5,
                9.5, 13.5, 8.0, 25.0, 12.5,
                7.5, 8.5, 12.5, 9.5, 32.0,
                7.0, 9.5, 11.5, 19.5, 35.0,
                7.5, 12.5, 11.5, 25.0, 65.0,
                15.5, 22.5, 10.5, 30.5, 20.5,
                6.5, 10.5, 215.0, 35.5, 85.0,
                4.5, 6.5, 6.0, 12.5, 15.5,
                6.5, 8.5, 9.5, 11.5, 35.5,
                7.5, 10.5, 13.5, 35.5, 15.5,
                8.5, 28.5, 35.5, 40.5, 12.5,
                42.5, 55.5, 45.5, 65.5, 125.0,
                45.5, 62.5, 42.5, 58.5, 155.0,
                45.5, 65.5, 48.5, 68.5, 185.0
            ],
            'mileage_kmpl': [
                22.5, 23.2, 21.5, 17.0, 19.5,
                20.5, 20.0, 16.8, 18.5, 18.0,
                20.0, 19.5, 17.5, 14.6, 14.0,
                15.0, 12.5, 20.0, 12.5, 15.2,
                18.5, 18.7, 17.8, 16.5, 9.5,
                19.5, 18.0, 17.1, 11.0, 10.0,
                18.5, 16.5, 14.5, 11.5, 20.9,
                13.5, 16.5, 15.5, 12.5, 13.0,
                20.0, 14.5, 8.5, 13.0, 8.0,
                22.0, 18.5, 20.0, 13.5, 16.0,
                18.5, 15.5, 16.5, 15.0, 8.5,
                17.5, 16.5, 15.0, 12.5, 18.0,
                15.5, 16.0, 15.5, 14.0, 17.5,
                17.0, 14.5, 15.8, 14.0, 9.5,
                16.5, 13.5, 16.2, 13.8, 8.5,
                14.5, 12.5, 15.0, 12.8, 7.5
            ],
            'engine_cc': [
                796, 1197, 1197, 1462, 1462,
                1197, 1197, 1353, 998, 1493,
                1199, 1199, 1199, 1956, 1956,
                1493, 2179, 1197, 2184, 2184,
                1199, 1199, 1498, 1199, 1997,
                1197, 1462, 1496, 2393, 2755,
                998, 1353, 1353, 2199, 1999,
                1451, 1499, 1349, 1996, 1451,
                999, 1329, 3799, 2488, 5552,
                999, 999, 999, 1330, 1330,
                1194, 1194, 1499, 1498, 2956,
                999, 1598, 1498, 1984, 1498,
                1598, 1984, 1984, 1984, 1498,
                1998, 1998, 1998, 1998, 2998,
                1984, 1984, 1984, 1984, 2998,
                1950, 1950, 1991, 1991, 2925
            ],
            'safety_rating': [
                3, 4, 4, 4, 4,
                4, 4, 5, 4, 4,
                4, 5, 5, 5, 5,
                3, 4, 4, 5, 4,
                4, 4, 4, 4, 5,
                4, 4, 4, 4, 5,
                4, 5, 4, 5, 5,
                5, 5, 5, 5, 5,
                4, 4, 5, 5, 5,
                4, 4, 4, 4, 4,
                4, 4, 4, 4, 5,
                4, 4, 4, 5, 4,
                4, 5, 5, 5, 4,
                5, 5, 5, 5, 5,
                5, 5, 5, 5, 5,
                5, 5, 5, 5, 5
            ],
            'seating_capacity': [
                5, 5, 5, 5, 7,
                5, 5, 5, 5, 5,
                5, 5, 5, 5, 7,
                7, 7, 5, 7, 4,
                5, 5, 5, 5, 7,
                5, 5, 5, 8, 7,
                5, 5, 7, 8, 5,
                5, 5, 5, 7, 6,
                5, 5, 2, 7, 8,
                5, 7, 5, 5, 5,
                5, 5, 5, 5, 7,
                5, 5, 5, 7, 5,
                5, 5, 5, 7, 5,
                5, 5, 5, 5, 5,
                5, 5, 5, 5, 5,
                5, 5, 5, 5, 5
            ]
        }
        
        self.cars_df = pd.DataFrame(cars_data)
        
    def prepare_models(self):
        """Prepare machine learning models"""
        # Encode categorical variables
        categorical_cols = ['brand', 'fuel_type', 'transmission', 'body_type']
        for col in categorical_cols:
            le = LabelEncoder()
            self.cars_df[f'{col}_encoded'] = le.fit_transform(self.cars_df[col])
            self.label_encoders[col] = le
        
        # Prepare features for clustering and similarity
        feature_cols = ['price_lakhs', 'mileage_kmpl', 'engine_cc', 'safety_rating', 
                       'seating_capacity', 'brand_encoded', 'fuel_type_encoded', 
                       'transmission_encoded', 'body_type_encoded']
        
        self.feature_matrix = self.cars_df[feature_cols].values
        self.feature_matrix_scaled = self.scaler.fit_transform(self.feature_matrix)
        
        # Train clustering model
        self.kmeans_model = KMeans(n_clusters=8, random_state=42)
        self.cars_df['cluster'] = self.kmeans_model.fit_predict(self.feature_matrix_scaled)
        
        # Calculate similarity matrix
        self.similarity_matrix = cosine_similarity(self.feature_matrix_scaled)
    
    def recommend_by_budget(self, min_budget, max_budget, fuel_type=None, body_type=None, num_recommendations=5):
        """Recommend cars based on budget and preferences"""
        filtered_cars = self.cars_df[
            (self.cars_df['price_lakhs'] >= min_budget) & 
            (self.cars_df['price_lakhs'] <= max_budget)
        ]
        
        if fuel_type and fuel_type != 'Any':
            filtered_cars = filtered_cars[filtered_cars['fuel_type'] == fuel_type]
        
        if body_type and body_type != 'Any':
            filtered_cars = filtered_cars[filtered_cars['body_type'] == body_type]
        
        if filtered_cars.empty:
            return None
        
        # Sort by safety rating and mileage
        filtered_cars = filtered_cars.sort_values(['safety_rating', 'mileage_kmpl'], ascending=[False, False])
        
        return filtered_cars.head(num_recommendations)
    
    def recommend_similar_cars(self, car_name, num_recommendations=5):
        """Recommend similar cars based on content similarity"""
        try:
            car_idx = self.cars_df[self.cars_df['model'].str.contains(car_name, case=False)].index[0]
            sim_scores = list(enumerate(self.similarity_matrix[car_idx]))
            sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
            sim_scores = sim_scores[1:num_recommendations+1]
            
            similar_indices = [i[0] for i in sim_scores]
            return self.cars_df.iloc[similar_indices]
        except IndexError:
            return None
    
    def recommend_by_usage(self, usage_type, budget_max=None, num_recommendations=5):
        """Recommend cars based on usage type"""
        recommendations = self.cars_df.copy()
        
        if budget_max:
            recommendations = recommendations[recommendations['price_lakhs'] <= budget_max]
        
        if usage_type == "City Driving":
            # Prioritize mileage and compact size
            recommendations = recommendations[recommendations['body_type'].isin(['Hatchback', 'Sedan'])]
            recommendations = recommendations.sort_values(['mileage_kmpl', 'price_lakhs'], ascending=[False, True])
        
        elif usage_type == "Family Travel":
            # Prioritize safety and seating
            recommendations = recommendations[recommendations['seating_capacity'] >= 5]
            recommendations = recommendations.sort_values(['safety_rating', 'seating_capacity'], ascending=[False, False])
        
        elif usage_type == "Off-Road Adventure":
            # Prioritize SUVs with good ground clearance
            recommendations = recommendations[recommendations['body_type'] == 'SUV']
            recommendations = recommendations.sort_values(['engine_cc', 'safety_rating'], ascending=[False, False])
        
        elif usage_type == "Luxury":
            # Prioritize premium brands and features
            luxury_brands = ['BMW', 'Audi', 'Mercedes-Benz']
            recommendations = recommendations[recommendations['brand'].isin(luxury_brands)]
            recommendations = recommendations.sort_values(['price_lakhs', 'safety_rating'], ascending=[False, False])
        
        elif usage_type == "Fuel Efficient":
            # Prioritize mileage
            recommendations = recommendations.sort_values(['mileage_kmpl', 'price_lakhs'], ascending=[False, True])
        
        return recommendations.head(num_recommendations)
    
    def get_best_cars_by_category(self):
        """Get best cars in different categories"""
        categories = {}
        
        # Best Budget Cars (Under 8 lakhs)
        budget_cars = self.cars_df[self.cars_df['price_lakhs'] <= 8]
        categories['Budget'] = budget_cars.nlargest(3, 'mileage_kmpl')
        
        # Best Luxury Cars (Above 40 lakhs)
        luxury_cars = self.cars_df[self.cars_df['price_lakhs'] >= 40]
        categories['Luxury'] = luxury_cars.nlargest(3, 'safety_rating')
        
        # Best SUVs
        suvs = self.cars_df[self.cars_df['body_type'] == 'SUV']
        categories['SUV'] = suvs.nlargest(3, 'safety_rating')
        
        # Most Fuel Efficient
        categories['Fuel Efficient'] = self.cars_df.nlargest(3, 'mileage_kmpl')
        
        return categories

class IndianCarRecommendationGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Indian Car Recommendation System by Shravani")
        self.root.geometry("1000x700")
        self.root.configure(bg='#1a1a2e')
        
        # Initialize recommendation system
        self.rec_system = IndianCarRecommendationSystem()
        
        self.setup_gui()
    
    def setup_gui(self):
        # Create main frame
        main_frame = tk.Frame(self.root, bg='#1a1a2e')
        main_frame.pack(fill=tk.BOTH, expand=True, padx=20, pady=20)
        
        # Title
        title_label = tk.Label(
            main_frame, 
            text="🚗 Indian Car Recommendation System by Shravani 🚗",
            font=('Arial', 20, 'bold'),
            fg='#ff6b35',
            bg='#1a1a2e'
        )
        title_label.pack(pady=(0, 20))
        
        # Create notebook for tabs
        self.notebook = ttk.Notebook(main_frame)
        self.notebook.pack(fill=tk.BOTH, expand=True)
        
        # Create tabs
        self.create_budget_tab()
        self.create_similar_tab()
        self.create_usage_tab()
        self.create_category_tab()
        
        # Style the notebook
        style = ttk.Style()
        style.theme_use('clam')
        style.configure('TNotebook', background='#16213e')
        style.configure('TNotebook.Tab', background='#0f3460', foreground='white', padding=[20, 10])
        style.map('TNotebook.Tab', background=[('selected', '#ff6b35')])
    
    def create_budget_tab(self):
        budget_frame = tk.Frame(self.notebook, bg='#16213e')
        self.notebook.add(budget_frame, text="Budget-Based")
        
        # Input section
        input_frame = tk.Frame(budget_frame, bg='#16213e')
        input_frame.pack(fill=tk.X, padx=20, pady=20)
        
        # Budget inputs
        budget_label = tk.Label(input_frame, text="Budget Range (in Lakhs):", 
                               font=('Arial', 12, 'bold'), fg='white', bg='#16213e')
        budget_label.grid(row=0, column=0, columnspan=2, sticky=tk.W, pady=(0, 10))
        
        tk.Label(input_frame, text="Min Budget:", font=('Arial', 10), 
                fg='white', bg='#16213e').grid(row=1, column=0, sticky=tk.W, padx=(0, 10))
        self.min_budget_entry = tk.Entry(input_frame, font=('Arial', 10), width=15, 
                                        bg='#0f3460', fg='white', insertbackground='white')
        self.min_budget_entry.grid(row=1, column=1, sticky=tk.W, pady=5)
        
        tk.Label(input_frame, text="Max Budget:", font=('Arial', 10), 
                fg='white', bg='#16213e').grid(row=2, column=0, sticky=tk.W, padx=(0, 10))
        self.max_budget_entry = tk.Entry(input_frame, font=('Arial', 10), width=15, 
                                        bg='#0f3460', fg='white', insertbackground='white')
        self.max_budget_entry.grid(row=2, column=1, sticky=tk.W, pady=5)
        
        # Filters
        tk.Label(input_frame, text="Fuel Type:", font=('Arial', 10), 
                fg='white', bg='#16213e').grid(row=3, column=0, sticky=tk.W, padx=(0, 10), pady=(10, 0))
        self.fuel_var = tk.StringVar(value="Any")
        fuel_combo = ttk.Combobox(input_frame, textvariable=self.fuel_var, 
                                 values=['Any', 'Petrol', 'Diesel', 'Electric'], width=12)
        fuel_combo.grid(row=3, column=1, sticky=tk.W, pady=(10, 5))
        
        tk.Label(input_frame, text="Body Type:", font=('Arial', 10), 
                fg='white', bg='#16213e').grid(row=4, column=0, sticky=tk.W, padx=(0, 10))
        self.body_var = tk.StringVar(value="Any")
        body_combo = ttk.Combobox(input_frame, textvariable=self.body_var, 
                                 values=['Any', 'Hatchback', 'Sedan', 'SUV', 'MPV'], width=12)
        body_combo.grid(row=4, column=1, sticky=tk.W, pady=5)
        
        tk.Button(input_frame, text="Get Recommendations", 
                 command=self.get_budget_recommendations,
                 bg='#ff6b35', fg='white', font=('Arial', 11, 'bold'), 
                 cursor='hand2', pady=8).grid(row=5, column=0, columnspan=2, pady=20)
        
        # Results section
        self.budget_results = scrolledtext.ScrolledText(budget_frame, height=18, 
                                                       bg='#0f3460', fg='white', font=('Arial', 10))
        self.budget_results.pack(fill=tk.BOTH, expand=True, padx=20, pady=(0, 20))
    
    def create_similar_tab(self):
        similar_frame = tk.Frame(self.notebook, bg='#16213e')
        self.notebook.add(similar_frame, text="Similar Cars")
        
        input_frame = tk.Frame(similar_frame, bg='#16213e')
        input_frame.pack(fill=tk.X, padx=20, pady=20)
        
        tk.Label(input_frame, text="Enter a car model you like:", 
                font=('Arial', 12, 'bold'), fg='white', bg='#16213e').pack(anchor=tk.W)
        
        self.car_entry = tk.Entry(input_frame, font=('Arial', 11), width=40, 
                                 bg='#0f3460', fg='white', insertbackground='white')
        self.car_entry.pack(pady=(10, 15), fill=tk.X)
        
        tk.Button(input_frame, text="Find Similar Cars", 
                 command=self.get_similar_recommendations,
                 bg='#4ecdc4', fg='white', font=('Arial', 11, 'bold'), 
                 cursor='hand2', pady=8).pack()
        
        self.similar_results = scrolledtext.ScrolledText(similar_frame, height=18, 
                                                        bg='#0f3460', fg='white', font=('Arial', 10))
        self.similar_results.pack(fill=tk.BOTH, expand=True, padx=20, pady=(0, 20))
    
    def create_usage_tab(self):
        usage_frame = tk.Frame(self.notebook, bg='#16213e')
        self.notebook.add(usage_frame, text="Usage-Based")
        
        input_frame = tk.Frame(usage_frame, bg='#16213e')
        input_frame.pack(fill=tk.X, padx=20, pady=20)
        
        tk.Label(input_frame, text="Select your primary usage:", 
                font=('Arial', 12, 'bold'), fg='white', bg='#16213e').pack(anchor=tk.W)
        
        self.usage_var = tk.StringVar(value="City Driving")
        usage_options = ["City Driving", "Family Travel", "Off-Road Adventure", "Luxury", "Fuel Efficient"]
        
        for option in usage_options:
            tk.Radiobutton(input_frame, text=option, variable=self.usage_var, value=option,
                          fg='white', bg='#16213e', selectcolor='#0f3460', 
                          font=('Arial', 10)).pack(anchor=tk.W, pady=2)
        
        tk.Label(input_frame, text="Maximum Budget (Lakhs, optional):", 
                font=('Arial', 10), fg='white', bg='#16213e').pack(anchor=tk.W, pady=(15, 5))
        
        self.usage_budget_entry = tk.Entry(input_frame, font=('Arial', 10), width=20, 
                                          bg='#0f3460', fg='white', insertbackground='white')
        self.usage_budget_entry.pack(anchor=tk.W, pady=(0, 15))
        
        tk.Button(input_frame, text="Get Usage-Based Recommendations", 
                 command=self.get_usage_recommendations,
                 bg='#45b7d1', fg='white', font=('Arial', 11, 'bold'), 
                 cursor='hand2', pady=8).pack()
        
        self.usage_results = scrolledtext.ScrolledText(usage_frame, height=16, 
                                                      bg='#0f3460', fg='white', font=('Arial', 10))
        self.usage_results.pack(fill=tk.BOTH, expand=True, padx=20, pady=(0, 20))
    
    def create_category_tab(self):
        category_frame = tk.Frame(self.notebook, bg='#16213e')
        self.notebook.add(category_frame, text="Best by Category")
        
        button_frame = tk.Frame(category_frame, bg='#16213e')
        button_frame.pack(fill=tk.X, padx=20, pady=20)
        
        tk.Label(button_frame, text="Explore the best cars in different categories:", 
                font=('Arial', 12, 'bold'), fg='white', bg='#16213e').pack(pady=(0, 15))
        
        tk.Button(button_frame, text="Show Best Cars by Category", 
                 command=self.get_category_recommendations,
                 bg='#96ceb4', fg='white', font=('Arial', 11, 'bold'), 
                 cursor='hand2', pady=10).pack()
        
        self.category_results = scrolledtext.ScrolledText(category_frame, height=18, 
                                                         bg='#0f3460', fg='white', font=('Arial', 10))
        self.category_results.pack(fill=tk.BOTH, expand=True, padx=20, pady=(0, 20))
    
    def get_budget_recommendations(self):
        try:
            min_budget = float(self.min_budget_entry.get()) if self.min_budget_entry.get() else 0
            max_budget = float(self.max_budget_entry.get()) if self.max_budget_entry.get() else 1000
            
            if min_budget > max_budget:
                messagebox.showwarning("Warning", "Minimum budget cannot be greater than maximum budget!")
                return
                
        except ValueError:
            messagebox.showwarning("Warning", "Please enter valid budget values!")
            return
        
        fuel_type = self.fuel_var.get() if self.fuel_var.get() != 'Any' else None
        body_type = self.body_var.get() if self.body_var.get() != 'Any' else None
        
        recommendations = self.rec_system.recommend_by_budget(
            min_budget, max_budget, fuel_type, body_type
        )
        
        self.budget_results.delete(1.0, tk.END)
        if recommendations is not None and not recommendations.empty:
            self.budget_results.insert(tk.END, f"🎯 Cars in your budget (₹{min_budget}L - ₹{max_budget}L):\n\n")
            
            for idx, (_, row) in enumerate(recommendations.iterrows(), 1):
                self.budget_results.insert(tk.END, f"{idx}. {row['brand']} {row['model']}\n")
                self.budget_results.insert(tk.END, f"   💰 Price: ₹{row['price_lakhs']:.1f} Lakhs\n")
                self.budget_results.insert(tk.END, f"   ⛽ Fuel: {row['fuel_type']} | Transmission: {row['transmission']}\n")
                self.budget_results.insert(tk.END, f"   🚗 Type: {row['body_type']} | Seats: {row['seating_capacity']}\n")
                self.budget_results.insert(tk.END, f"   📊 Mileage: {row['mileage_kmpl']} kmpl | Safety: {row['safety_rating']}/5\n")
                self.budget_results.insert(tk.END, f"   🔧 Engine: {row['engine_cc']}cc\n\n")
        else:
            self.budget_results.insert(tk.END, "❌ No cars found matching your criteria. Try adjusting your filters.")
    
    def get_similar_recommendations(self):
        car_name = self.car_entry.get().strip()
        if not car_name:
            messagebox.showwarning("Warning", "Please enter a car model!")
            return
        
        recommendations = self.rec_system.recommend_similar_cars(car_name)
        
        self.similar_results.delete(1.0, tk.END)
        if recommendations is not None and not recommendations.empty:
            self.similar_results.insert(tk.END, f"🔍 Cars similar to '{car_name}':\n\n")
            
            for idx, (_, row) in enumerate(recommendations.iterrows(), 1):
                self.similar_results.insert(tk.END, f"{idx}. {row['brand']} {row['model']}\n")
                self.similar_results.insert(tk.END, f"   💰 Price: ₹{row['price_lakhs']:.1f} Lakhs\n")
                self.similar_results.insert(tk.END, f"   ⛽ Fuel: {row['fuel_type']} | Transmission: {row['transmission']}\n")
                self.similar_results.insert(tk.END, f"   🚗 Type: {row['body_type']} | Seats: {row['seating_capacity']}\n")
                self.similar_results.insert(tk.END, f"   📊 Mileage: {row['mileage_kmpl']} kmpl | Safety: {row['safety_rating']}/5\n")
                self.similar_results.insert(tk.END, f"   🔧 Engine: {row['engine_cc']}cc\n\n")
        else:
            self.similar_results.insert(tk.END, f"❌ No similar cars found for '{car_name}'. Please try a different model name.")
    
    def get_usage_recommendations(self):
        usage_type = self.usage_var.get()
        budget_max = None
        
        if self.usage_budget_entry.get():
            try:
                budget_max = float(self.usage_budget_entry.get())
            except ValueError:
                messagebox.showwarning("Warning", "Please enter a valid budget!")
                return
        
        recommendations = self.rec_system.recommend_by_usage(usage_type, budget_max)
        
        self.usage_results.delete(1.0, tk.END)
        if not recommendations.empty:
            self.usage_results.insert(tk.END, f"🎯 Best cars for {usage_type}:\n\n")
            
            for idx, (_, row) in enumerate(recommendations.iterrows(), 1):
                self.usage_results.insert(tk.END, f"{idx}. {row['brand']} {row['model']}\n")
                self.usage_results.insert(tk.END, f"   💰 Price: ₹{row['price_lakhs']:.1f} Lakhs\n")
                self.usage_results.insert(tk.END, f"   ⛽ Fuel: {row['fuel_type']} | Transmission: {row['transmission']}\n")
                self.usage_results.insert(tk.END, f"   🚗 Type: {row['body_type']} | Seats: {row['seating_capacity']}\n")
                self.usage_results.insert(tk.END, f"   📊 Mileage: {row['mileage_kmpl']} kmpl | Safety: {row['safety_rating']}/5\n")
                self.usage_results.insert(tk.END, f"   🔧 Engine: {row['engine_cc']}cc\n\n")
        else:
            self.usage_results.insert(tk.END, "❌ No cars found for your usage type and budget.")
    
    def get_category_recommendations(self):
        categories = self.rec_system.get_best_cars_by_category()
        
        self.category_results.delete(1.0, tk.END)
        self.category_results.insert(tk.END, "🏆 BEST CARS BY CATEGORY 🏆\n\n")
        
        category_icons = {
            'Budget': '💰',
            'Luxury': '👑', 
            'SUV': '🚙',
            'Fuel Efficient': '⛽'
        }
        
        for category, cars in categories.items():
            icon = category_icons.get(category, '🚗')
            self.category_results.insert(tk.END, f"{icon} {category.upper()} CARS:\n")
            self.category_results.insert(tk.END, "=" * 40 + "\n")
            
            for idx, (_, row) in enumerate(cars.iterrows(), 1):
                self.category_results.insert(tk.END, f"{idx}. {row['brand']} {row['model']}\n")
                self.category_results.insert(tk.END, f"   💰 Price: ₹{row['price_lakhs']:.1f} Lakhs\n")
                self.category_results.insert(tk.END, f"   ⛽ Fuel: {row['fuel_type']} | Type: {row['body_type']}\n")
                self.category_results.insert(tk.END, f"   📊 Mileage: {row['mileage_kmpl']} kmpl | Safety: {row['safety_rating']}/5\n\n")
            
            self.category_results.insert(tk.END, "\n")

def main():
    root = tk.Tk()
    app = IndianCarRecommendationGUI(root)
    root.mainloop()

if __name__ == "__main__":
    main()